In [ ]:
import re
import rhinoMorph
import pandas as pd
from tqdm import tqdm
from pykospacing import Spacing
from sklearn.feature_extraction.text import TfidfVectorizer

tqdm.pandas()                  # tqdm과 pandas 통합

rn = rhinoMorph.startRhino()   # Rhino 객체 초기화

In [ ]:
spacing = Spacing()

def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# 불용어 리스트 가져오기
def get_stopwords():
    stopwords = set()
    with open('../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
        for line in file:
            stopwords.add(line.strip())
    return stopwords

# 형태소 분석 및 불용어 제거 함수
def morph_and_remove_stopwords(text, stopwords):
    text = preprocessing(text)
    morphs = rhinoMorph.onlyMorph_list(rn, text)
    meaningful_words = [word for word in morphs if word not in stopwords]
    return meaningful_words


In [ ]:
# 불용어 리스트 가져오기
stopwords = get_stopwords()

df = pd.read_csv(r'..\..\..\data\combined_questions.csv', encoding='cp949')

# 데이터프레임의 'feature' 열에 대해 형태소 분석 및 불용어 제거 적용
df['processed_feature'] = df['feature'].apply(lambda x: morph_and_remove_stopwords(x, stopwords))

In [ ]:
# 리스트를 문자열로 변환
df['processed_feature_str'] = df['processed_feature'].apply(lambda x: ' '.join(x))

# TF-IDF 벡터화 적용
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_feature_str'])

# 데이터프레임으로 변환
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# 각 문서에서 상위 TF-IDF 값을 가지는 단어 추출하여 새로운 데이터프레임 생성
top_n = 5  # 상위 n개 단어
top_tfidf_list = []

for i in range(tfidf_df.shape[0]):
    sorted_indices = tfidf_df.iloc[i].sort_values(ascending=False).index[:top_n]
    sorted_values = tfidf_df.iloc[i][sorted_indices]
    top_tfidf_dict = {f'term_{j+1}': sorted_indices[j] for j in range(top_n)}
    top_tfidf_dict.update({f'tfidf_{j+1}': sorted_values.values[j] for j in range(top_n)})
    top_tfidf_list.append(top_tfidf_dict)

# 상위 TF-IDF 값을 가지는 단어와 값을 포함한 데이터프레임 생성
top_tfidf_df = pd.DataFrame(top_tfidf_list)